In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import metrics

In [11]:
training_data = pd.read_excel("C:/Vaaman/Finance model predictor/processed_data_20dec.xlsx")
training_data['class'] = training_data['7 day_future_percentage_increase'].apply(lambda x: (x > 2.0) *1)

In [3]:
col_list = training_data.columns

In [4]:
col_list

Index(['Ref Date', 'Intial', '7 day_past_min',
       '7 day_past_percentage_change_min', '7 day_past_max',
       '7 day_past_percentage_change_max', '14 day_past_min',
       '14 day_past_percentage_change_min', '14 day_past_max',
       '14 day_past_percentage_change_max', '21 day_past_min',
       '21 day_past_percentage_change_min', '21 day_past_max',
       '21 day_past_percentage_change_max', '28 day_past_min',
       '28 day_past_percentage_change_min', '28 day_past_max',
       '28 day_past_percentage_change_max', '7 day_future_max',
       '7 day_future_percentage_increase'],
      dtype='object')

In [12]:
x = training_data[['7 day_past_percentage_change_min','7 day_past_percentage_change_max','14 day_past_percentage_change_min','14 day_past_percentage_change_max','21 day_past_percentage_change_min','21 day_past_percentage_change_max','28 day_past_percentage_change_min','28 day_past_percentage_change_max']]  
y = training_data[['class']]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 42)

In [14]:
model = Sequential([
    Dense(units = 64, activation = 'relu'),
    Dense(units = 32, activation = 'relu'),
    Dense(units = 32, activation = 'relu'),
    Dense(units = 16, activation = 'relu'),
    Dense(units = 16, activation = 'relu'),
    Dense(units = 8, activation = 'relu'),
    Dense(units = 8, activation = 'relu'),
    Dense(units = 4, activation = 'relu'),
    Dense(units = 4, activation = 'relu'),
    Dense(units = 1, activation = 'sigmoid'),
])

In [18]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.compile(optimizer = Adam(0.001), 
             loss = tf.keras.losses.BinaryCrossentropy(),
             metrics=['accuracy', metrics.Precision(), metrics.Recall()])


In [62]:
print(x_test.shape)

(51, 1)


In [19]:
model.fit(x_train, y_train, epochs = 100,batch_size = 16, validation_data=(x_test, y_test))

Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 13s 135ms/step - accuracy: 0.4597 - loss: 0.6978 - precision: 0.3731 - recall: 0.8650 - val_accuracy: 0.7059 - val_loss: 0.6876 - val_precision: 0.6000 - val_recall: 0.1875
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.6729 - loss: 0.6865 - precision: 0.6777 - recall: 0.1356 - val_accuracy: 0.7059 - val_loss: 0.6816 - val_precision: 1.0000 - val_recall: 0.0625
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6921 - loss: 0.6808 - precision: 0.5000 - recall: 0.0144 - val_accuracy: 0.7059 - val_loss: 0.6768 - val_precision: 1.0000 - val_recall: 0.0625
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.6890 - loss: 0.6758 - precision: 0.2857 - recall: 0.0080 - val_accuracy: 0.6863 - val_loss: 0.6716 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6516 - loss: 0.6744 - precision: 0.7143 - recall: 0.0184 - val_accuracy: 0.6863 - va

In [20]:
loss, accuracy, precision,recall = model.evaluate(x_train, y_train)
print ( loss, accuracy, precision,recall)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8133 - loss: 0.3830 - precision: 0.7163 - recall: 0.6734
0.35533514618873596 0.8399999737739563 0.761904776096344 0.7384615540504456


In [21]:
loss, accuracy, precision,recall = model.evaluate(x_test, y_test)
print ( loss, accuracy, precision,recall)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6266 - loss: 0.7808 - precision: 0.3997 - recall: 0.5417
0.7984581589698792 0.6274510025978088 0.42105263471603394 0.5


In [68]:
x_train = np.array(x_train).flatten()  
y_train = np.array(y_train).flatten()  
probabilities = model.predict(x_train)
probabilities = np.array(probabilities).flatten()
df = pd.DataFrame({
    'X_Train': x_train,        
    'Y_Train': y_train,        
    'Probability': probabilities
})
with pd.ExcelWriter("processed_data.xlsx", mode='a', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='TrainScring', index=False)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [214]:
print(x_test)


[-3.61255613e-03  7.64814401e-01  2.75696769e-01  3.11626089e+00
  5.89256259e+00 -2.15717518e-01  8.91628879e-01 -2.14129094e-03
  1.53368221e+00  3.37539451e+00  9.20683147e-01  1.58390334e+00
  3.63997034e+00  1.04821684e+00  2.86524430e+00  7.23611408e+00
  1.56811950e+00  1.50490607e+01  3.53105008e-01  7.41819740e+00
 -1.90021446e+00  7.86102633e+00  1.27276362e+00  2.33530076e+00
  5.92691037e-01 -1.20241167e-01  1.44154358e+00  1.45111920e+00
  1.67614954e+00  9.34555971e-01  5.86435186e-01  2.92127088e+00
  1.53753040e+00  1.73702998e+00 -4.24868643e-01 -1.90843229e+00
  5.57702989e+00  3.04301675e+00  2.98593263e+00  1.70819515e+00
  2.10999070e+00 -1.87659426e-01  5.88067882e-01  1.24887471e+00
  1.60930161e+00  1.31098091e+01  4.03577293e+00  4.55428611e+00
  2.24052366e-01  2.36116637e+00  1.44376647e+00]


In [35]:
p = model.predict(x_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


In [69]:
u = model.predict(x_train)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [37]:
print(model.input_shape)

(None, 1)


In [42]:
print(x_train.shape)

(200, 1)


In [39]:
print(x_train.shape)

(200, 1)


In [ ]:
def calc_prec_recall(model, dataset, threshold):
    '''
    '''
    df = dataset.copy()
    tot_ones = df[df['ground_truth'] == 1].shape(0)
    tot_zeros = df[df['ground truth'] == 0].shape(0)
    df['prediction'] = df[df[p]]